<a href="https://colab.research.google.com/github/iam-nhi-nguyen/Massp2019/blob/master/analysis/pso_markovian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Penalty Shootouts Analysis - Markovian model analysis

*Nhi Nguyen*



In [1]:
# libraries
import pandas as pd
import numpy as np
import sympy
import math

# 2 rounds

Proof of concept: Complete the formula for the first 2 rounds

In [2]:
nround = 2

### Set up sequences

* **Step 1**: Create $2^{2 \times nround + 1} - 1$ binary sequences of penalty outcomes of lengths $\{0, 1, \cdots, 2 \times nround\}$

* **Step 2**: Create 2 functions `o2d()` (**o**utcomes **to** **d**ifferences) and `d2o()` to map from sequences of penalty outcomes to sequences of goal differences and vice versa

* **Step 3**: Use the previos functions to create 2 dictionaries `out2diff` and `diff2out` top map the $2^{2 \times nround + 1} - 1$ sequences of penalty outcomes to the corresponding sequences of goal differences and vice versa

* **Step 4**: Determine `complete_paths` (i.e. sequences where the penalty shootout should terminate) and remove invalid sequences (i.e. sequences where the penalty shootout should have ended earlier)

In [3]:
# Step 1: Create 2^0 + 2^1 + 2^2 + 2^3 + 2^4 = 31 binary sequences

origin = ()
penalty_outcomes = [origin]
prev_seqs = [origin]
for k in range(2 * nround):
    # prev_seqs currently have all binary sequences of length k
    # use prev_seqs to create curr_seqs with all binary sequences of length k+1
    curr_seqs = []
    for p in prev_seqs:
        curr_seqs.append(p + (0,))
        curr_seqs.append(p + (1,))
    penalty_outcomes += curr_seqs
    prev_seqs = curr_seqs

print(f"The number of sequences of outcomes is {len(penalty_outcomes)}.")

The number of sequences of outcomes is 31.


In [4]:
# Step 2: Create o2d() and d2o()

def o2d(x):
    if len(x) == 0:
        return x
    ret = x[:1]
    for i in range(1, len(x)):
        ## if i % 2:   # team 2
        ##     ret = ret + (ret[-1] - x[i],)
        ## else:       # team 1
        ##     ret = ret + (ret[-1] + x[i],)
        ## possible combined formula
        ret = ret + (ret[-1] + x[i] * (1-2*(i%2)),)
    return ret

def d2o(x):
    if len(x) == 0:
        return x
    x = (0,) + x
    ret = ()
    for i in range(1, len(x)):
        if i % 2:   # team 1
            if x[i] == x[i-1]:
                ret = ret + (0,)
            elif x[i] == x[i-1] + 1:
                ret = ret + (1,)
            else:
                raise ValueError(f"Invalid sequence of goal differences: {x}")
        else:       # team 2
            if x[i] == x[i-1]:
                ret = ret + (0,)
            elif x[i] == x[i-1] - 1:
                ret = ret + (1,)
            else:
                raise ValueError(f"Invalid sequence of goal differences: {x}")
        # think about combined formula
    return ret

print("Testing functions o2d() and d2o()")
print(f"Outcomes (1, 0, 1, 1) is equivalence to differences {o2d((1, 0, 1, 1))}.")
print(f"Differences (1, 1, 2, 1) is equivalence to outcomes {d2o((1, 1, 2, 1))}.")

Testing functions o2d() and d2o()
Outcomes (1, 0, 1, 1) is equivalence to differences (1, 1, 2, 1).
Differences (1, 1, 2, 1) is equivalence to outcomes (1, 0, 1, 1).


In [5]:
# Step 3: Create dictionaries out2diff and diff2out
# using o2d() and penalty_outcomes

out2diff = {}
diff2out = {}
for out in penalty_outcomes:
    diff = o2d(out)
    out2diff[out] = diff
    diff2out[diff] = out

print("Testing dictionaries out2diff and diff2out")
print(f"Outcomes (1, 0, 1, 1) is equivalence to differences {out2diff[(1, 0, 1, 1)]}.")
print(f"Differences (1, 1, 2, 1) is equivalence to outcomes {diff2out[(1, 1, 2, 1)]}.")

Testing dictionaries out2diff and diff2out
Outcomes (1, 0, 1, 1) is equivalence to differences (1, 1, 2, 1).
Differences (1, 1, 2, 1) is equivalence to outcomes (1, 0, 1, 1).


In [6]:
# Step 4: Find complete paths and remove invalid sequences

complete_paths = set()
ended = set()           # sequences that should have endeds
for diff in sorted(list(diff2out.keys()), key=len): # examine longer sequences first
    k = len(diff)
    if k == 0:
        continue
    # check for paths that should have ended earlier
    if k > 0 and (diff[:-1] in ended):
        out = diff2out[diff]
        out2diff.pop(out)
        diff2out.pop(diff)
        ended.add(diff)
        continue
    # check for complete paths
    if k == 2*nround:               # max length
        complete_paths.add(diff)
        ended.add(diff)
    elif k % 2:                     # last shot is from team 1
        max_s = diff[-1] + (nround - k//2 - 1)  # max score difference after nround
        min_s = diff[-1] - (nround - k//2)      # min score difference after nround
        if max_s * min_s > 0:   # the result is the same regardless
            complete_paths.add(diff)
            ended.add(diff)
    else:                           # last shot is from team 2
        max_s = diff[-1] + (nround - k//2)      # max score difference after nround
        min_s = diff[-1] - (nround - k//2)      # min score difference after nround
        if max_s * min_s > 0:   # the result is the same regardless
            complete_paths.add(diff)
            ended.add(diff)

# There should be 4 invalid sequences and 14 complete paths
print(f"The number of valid sequences is {len(penalty_outcomes) - len(diff2out)}.")
print(f"The number of complete paths is {len(complete_paths)}.")

The number of valid sequences is 4.
The number of complete paths is 14.


### Set up network

* **Step 1**: Create 2 functions `r2s()` (**r**ound **to** **s**hot) and `s2r()` to map from state $\{i, t\}$ to shot number $k$ and vice versa ($i \in \{1, 2\}$ represents the current team and $t$ represents the current round

* **Step 2**: Set up 2 dictionaries `next_nodes` and `prev_nodes` to represent the network, where 
    * if we are currently at $(k,s)$, the path travel to `next_nodes[(k,s)][1]` if the next shot is $1$ (is a goal); otherwise, the path travel to `next_nodes[(k,s)][0]`; if the penalty shootout is supposed to end at $(k,s)$ then `next_nodes[(k,s)] = [None, None]`
    * if we are currently at $(k,s)$ and the latest shot is $1$ (is a goal), then the previous node in this path is `prev_nodes[(k,s)][1]`; otherwise, the previous node in this path is `prev_nodes[(k,s)][0]`; if such path doesn't exist (e.g. there is no path to $(k,s)$ where the last shot is $1$) then the corresponding `prev_nodes[(k,s)]` is `None`

In [7]:
# Step 1: Create r2s() and s2r()

def r2s(i, t):
    return i + (t - 1) * 2

def s2r(k):
    if k == 0: # special case when started
        return 0, 0
    i = 2 - k % 2
    t = (k - i) // 2 + 1
    return i, t

print("Testing functions r2s() and s2r()")
print(f"State (2, 1) is equivalence to shot {r2s(2, 1)}.")
print(f"Shot 2 is equivalence to state {s2r(2)}.")

Testing functions r2s() and s2r()
State (2, 1) is equivalence to shot 2.
Shot 2 is equivalence to state (2, 1).


In [8]:
# Step 2: Set up the network using subscript (k, s)
# where k is the shot number and s is the current score difference

start = (0, 0)
nodes = set([start])            # a set contains all possible nodes
term_nodes = set()              # a set contains all terminating nodes
next_nodes = {start: [None]*2}
prev_nodes = {start: [None]*2}
for path in complete_paths:
    for k in range(len(path)):
        s = path[k]
        node = (k+1, s)
        nodes.add(node)
        prev_nodes[node] = [None]*2
        next_nodes[node] = [None]*2
        if k + 1 == len(path):      # terminating node
            term_nodes.add(node)

for node in nodes:
    k, s = node
    # populate next_nodes
    if node not in term_nodes:
        if (k+1, s) in nodes:
            next_nodes[(k, s)][0] = (k+1, s)
        ## if k % 2:                       # next team is team 2
        ##     if (k+1, s-1) in nodes:   
        ##         next_nodes[(k, s)][1] = (k+1, s-1)
        ## else:                           # next team is team 1
        ##      if (k+1, s+1) in nodes:   
        ##         next_nodes[(k, s)][1] = (k+1, s+1)
        ## possible combine formula
        if (k+1, s +  (1-2*(k%2))) in nodes:
            next_nodes[(k, s)][1] = (k+1, s +  (1-2*(k%2)))
    # populate prev_nodes
    if (k-1, s) in nodes and (k-1, s) not in term_nodes:
        prev_nodes[(k,s)][0] = (k-1, s)
    if (k-1, s +  (1-2*(k%2))) in nodes  and (k-1, s +  (1-2*(k%2))) not in term_nodes:
        prev_nodes[(k, s)][1] = (k-1, s +  (1-2*(k%2)))

# There should be 13 nodes
print(f"The number of nodes is {len(nodes)}.")

# A quick check to make sure that a in next[b] iff b in prev[a]
# If the condition is true, nothing should be printed
for node in next_nodes.keys():
    for next in next_nodes[node]:
        if next and node not in prev_nodes[next]:
            print(node, next)

for prev in prev_nodes.keys():
    for prev in prev_nodes[node]:
        if prev and node not in next_nodes[prev]:
            print(node, prev)

The number of nodes is 13.


In [9]:
prev_nodes

{(0, 0): [None, None],
 (1, 0): [(0, 0), None],
 (1, 1): [None, (0, 0)],
 (2, -1): [None, (1, 0)],
 (2, 0): [(1, 0), (1, 1)],
 (2, 1): [(1, 1), None],
 (3, -1): [(2, -1), None],
 (3, 0): [(2, 0), (2, -1)],
 (3, 1): [(2, 1), (2, 0)],
 (3, 2): [None, (2, 1)],
 (4, -1): [None, (3, 0)],
 (4, 0): [(3, 0), (3, 1)],
 (4, 1): [(3, 1), None]}

### Set up variables and expression using `sympy`

* **Step 1**: Create symbols for $n(d_1,\cdots)$ for all valid sequences of goal differences

* **Step 2**: Create symbols for the transition probabilities $q_{i,t}(s)$

* **Step 3**: Express the transition probability $q_{i,t}(s)$ and its complement $(1 - q_{i,t}(s))$ in terms of $n(d_1, \cdots)$

In [10]:
# Step 1: Create symbols for n(d1, ...)

n_symbol = {}
for diff in diff2out.keys():
    # name in format n(d1;d2;...) where d1, d2 are number
    name = "n" + str(diff).replace(" ","").replace(",",";")
    n_symbol[diff] = sympy.symbols(name)
    
# special cases
n_symbol[()] = sympy.symbols("n")       
n_symbol[(0,)] = sympy.symbols("n(0)")
n_symbol[(1,)] = sympy.symbols("n(1)")

# Example of an expression using these symbols
# e.g. we want to get the expression (n(1,0,1) + n(0,0,1))/(n(1,0) + n(0,0))
expr = (n_symbol[(1,0,1)] + n_symbol[(0,0,1)]) / (n_symbol[(1,0)] + n_symbol[(0,0)])
expr

(n(0;0;1) + n(1;0;1))/(n(0;0) + n(1;0))

In [11]:
# Step 2: Create symbols for q_{i,t}(s)

q_symbol = {}
for node in nodes:
    if node in term_nodes:
        continue
    k, s = node
    i, t = s2r(k+1) # s is the goal difference at shot k but (i,t) represents the next shot
    # name in the format q_{i,t}(s)
    name = "q_{" + f"{i};{t}" + "}" + f"({s})"
    q_symbol[(i, t, s)] = sympy.symbols(name)

# The number of transitions should be 8
print(f"The number of transitions is {len(q_symbol)}")

# Example of an expression using these symbols
# e.g. we want to get the expression q_{1,1}(0) * (1 - q_{2,1}(1))
expr = q_symbol[(1,1,0)] * (1 - q_symbol[(2,1,1)])
expr

The number of transitions is 8


q_{1;1}(0)*(1 - q_{2;1}(1))

In [12]:
# Step 3: Express q's in terms of n's
# In particular, q_expr[(i,t,s)][1] = q_{i,t}(s) 
# and q_expr[(i,t,s)][0] = 1 - q_{i,t}(s)

q_expr = {}
for i, t, s in q_symbol.keys():
    k = r2s(i,t) - 1 # previous shot
    if k == 0:
        if s == 0:
            q_expr[(i,t,s)] = [n_symbol[(0,)] / n_symbol[()], n_symbol[(1,)] / n_symbol[()]]
        else:
            raise ValueError(f"Invalid state (i, t, s) = ({i}, {t}, {s})")
    else:
        denom = 0       # add up all n(.,.,s) of length k
        num_compl = 0   # add up all n(.,.,s,0)
        num = 0         # add up all n(.,.,s,s+3-2*i) depending on k
        for seq in n_symbol.keys():
            if len(seq) == k and seq[-1] == s:
                denom += n_symbol[seq]
            elif len(seq) == k + 1 and seq[-2] == s:
                if seq[-1] == s:
                    num_compl += n_symbol[seq]
                elif seq[-1] == s + 3-2*i:
                    num += n_symbol[seq]
        # populate q_expr
        q_expr[(i,t,s)] = [num_compl / denom, num / denom]

# Display all q_expr
for state in sorted(q_symbol.keys(), key=lambda x: (x[1], x[0], -x[2])):
    display(sympy.Eq(q_symbol[state], q_expr[state][1]))
    print()
    display(sympy.Eq(1 - q_symbol[state], q_expr[state][0]))
    print("*"*10)

Eq(q_{1;1}(0), n(1)/n)

Eq(1 - q_{1;1}(0), n(0)/n)

**********


Eq(q_{2;1}(1), n(1;0)/n(1))

Eq(1 - q_{2;1}(1), n(1;1)/n(1))

...

Eq(1 - q_{2;2}(1), (n(0;0;1;1) + n(1;0;1;1) + n(1;1;1;1))/(n(0;0;1) + n(1;0;1) + n(1;1;1)))

**********


Eq(q_{2;2}(0), (n(0;-1;0;-1) + n(0;0;0;-1) + n(1;0;0;-1))/(n(0;-1;0) + n(0;0;0) + n(1;0;0)))

Eq(1 - q_{2;2}(0), (n(0;-1;0;0) + n(0;0;0;0) + n(1;0;0;0))/(n(0;-1;0) + n(0;0;0) + n(1;0;0)))

**********


### Express formula for true probability, Markov probability, and adjustment

* **Step 1**: Create symbols for true probability $P(d_i = s_i, d_j = s_j)$, Markovian probability $Pr(d_i = s_i, d_j = s_i)$, and adjustment $A(d_i = s_i, d_j = s_j)$ 

* **Step 2**: Use $i = 0$ and $s_i = 0$ (i.e. $d_0 = 0$),
    * **Step 2.1**: Express the formulas for the true probability
    * **Step 2.2**: Express the formulas for the Markovian probability
    * **Step 2.3**: Express the formulas for the adjustment

    We should have $$P(d_i = s_i, d_j = s_j) = Pr(d_i = s_i, d_j = s_i) + A(d_i = s_i, d_j = s_j)$$

* **Step 3**: Express the all formulas for the previous variables

In [13]:
# Step 1: Create the probability symbol

p_symbol = {}
pr_symbol = {}
a_symbol = {}

for start in nodes:
    for end in nodes:
        if end[0] < start[0]:   # start = (i, s_i); end = (j, s_j); must have i < j
            continue
        if end == start or end[0] > start[0]:
            i, si = start
            j, sj = end
            # condition in the format (d_i = s_i, d_j = s_j)
            cond = "(d_{" + f"{i}" + "}" + f"={si};" + "d_{" + f"{j}" + "}" + f"={sj})"
            # store the result in key ((i, s_i), (j, s(j))
            p_symbol[((i, si), (j, sj))] = sympy.symbols(f"P{cond}")
            pr_symbol[((i, si), (j, sj))] = sympy.symbols(f"Pr{cond}")
            a_symbol[((i, si), (j, sj))] = sympy.symbols(f"A{cond}")

print(f"The number of different conditions is {len(p_symbol)}.")

The number of different conditions is 78.


In [14]:
# Step 2.0: Get all possible end nodes given the start node d_0 = 0
start = (0,0)

all_paths = {}
for end in sorted(nodes): # use BFS
    if end[0] < start[0]:   # start = (i, s_i); end = (j, s_j); must have i < j
        continue
    if end[0] == start[0]:
        if end == start:    # base case: when end = start
            all_paths[(start, end)] = [()]
        continue
    paths = []
    for prev in prev_nodes[end]: # recursive case
        if prev is None:
            continue
        for sub_path in all_paths[(start, prev)]:
            paths.append(sub_path + end[1:])
    all_paths[(start, end)] = paths

# The number of end nodes should be the same as the number of all nodes (13)
print(f"The number of ends is {len(all_paths)}.")
# The number of paths/subpaths from the beginning should be 27
print(f"The number of paths starting from d_0 = 0 is {sum(list(map(len, all_paths.values())))}.")

The number of ends is 13.
The number of paths starting from d_0 = 0 is 27.


In [15]:
# Step 2.1: Get the formula for the true probability

p_expr = {}

for cond in all_paths.keys(): # cond = (start, end)
    expr = 0
    for path in all_paths[cond]: # simply add up n() for all paths
        expr += n_symbol[path]
    p_expr[cond] = sympy.simplify(expr / n_symbol[()]) # divide by n total

for cond in all_paths.keys():
    display(sympy.Eq(p_symbol[cond], p_expr[cond]))
    print()

Eq(P(d_{0}=0;d_{0}=0), 1)

Eq(P(d_{0}=0;d_{1}=0), n(0)/n)

Eq(P(d_{0}=0;d_{1}=1), n(1)/n)

Eq(P(d_{0}=0;d_{2}=-1), n(0;-1)/n)

Eq(P(d_{0}=0;d_{2}=0), (n(0;0) + n(1;0))/n)

...

Eq(P(d_{0}=0;d_{3}=1), (n(0;0;1) + n(1;0;1) + n(1;1;1))/n)

Eq(P(d_{0}=0;d_{3}=2), n(1;1;2)/n)

Eq(P(d_{0}=0;d_{4}=-1), (n(0;-1;0;-1) + n(0;0;0;-1) + n(1;0;0;-1))/n)

Eq(P(d_{0}=0;d_{4}=0), (n(0;-1;0;0) + n(0;0;0;0) + n(0;0;1;0) + n(1;0;0;0) + n(1;0;1;0) + n(1;1;1;0))/n)

Eq(P(d_{0}=0;d_{4}=1), (n(0;0;1;1) + n(1;0;1;1) + n(1;1;1;1))/n)

In [16]:
# Step 2.2: Get the formula for the Markov probability

pr_expr = {}

for end in sorted(nodes): # use BFS
    if end[0] < start[0]:   # start = (i, s_i); end = (j, s_j); must have i < j
        continue
    if end[0] == start[0]:
        if end == start:    # base case: when end = start
            pr_expr[(start, end)] = 1
        continue
    expr = 0
    for prev in prev_nodes[end]:
        if prev is None:
            continue
        k, s = prev
        i, t = s2r(k+1)
        # Using Markov transition probability
        expr += pr_expr[(start, prev)] * q_expr[(i,t,s)][end[1] != s]
    pr_expr[(start, end)] = sympy.simplify(expr)

for cond in all_paths.keys():
    display(sympy.Eq(pr_symbol[cond], pr_expr[cond]))
    print()

Eq(Pr(d_{0}=0;d_{0}=0), 1)

Eq(Pr(d_{0}=0;d_{1}=0), n(0)/n)

Eq(Pr(d_{0}=0;d_{1}=1), n(1)/n)

Eq(Pr(d_{0}=0;d_{2}=-1), n(0;-1)/n)

Eq(Pr(d_{0}=0;d_{2}=0), (n(0;0) + n(1;0))/n)

...

Eq(Pr(d_{0}=0;d_{3}=1), (n(0;0;1) + n(1;0;1) + n(1;1;1))/n)

Eq(Pr(d_{0}=0;d_{3}=2), n(1;1;2)/n)

Eq(Pr(d_{0}=0;d_{4}=-1), (n(0;-1;0;-1) + n(0;0;0;-1) + n(1;0;0;-1))/n)

Eq(Pr(d_{0}=0;d_{4}=0), (n(0;-1;0;0) + n(0;0;0;0) + n(0;0;1;0) + n(1;0;0;0) + n(1;0;1;0) + n(1;1;1;0))/n)

Eq(Pr(d_{0}=0;d_{4}=1), (n(0;0;1;1) + n(1;0;1;1) + n(1;1;1;1))/n)

In [17]:
# Step 2.3: Get the formula for the adjustment

a_expr = {}
for cond in all_paths.keys():
    a_expr[cond] = sympy.simplify(p_expr[cond] - pr_expr[cond])

for cond in all_paths.keys():
    display(sympy.Eq(a_symbol[cond], a_expr[cond]))
    print()

Eq(A(d_{0}=0;d_{0}=0), 0)

Eq(A(d_{0}=0;d_{1}=0), 0)

Eq(A(d_{0}=0;d_{1}=1), 0)

Eq(A(d_{0}=0;d_{2}=-1), 0)

Eq(A(d_{0}=0;d_{2}=0), 0)

...

Eq(A(d_{0}=0;d_{3}=1), 0)

Eq(A(d_{0}=0;d_{3}=2), 0)

Eq(A(d_{0}=0;d_{4}=-1), 0)

Eq(A(d_{0}=0;d_{4}=0), 0)

Eq(A(d_{0}=0;d_{4}=1), 0)

In [18]:
# Step 3: Repeat the same process for all other start node

for start in nodes:
    
    # all paths
    cond_paths = {}
    for end in sorted(nodes): # use BFS
        if end[0] < start[0]:  
            continue
        if end[0] == start[0]:
            if end == start:    
                cond_paths[(start, end)] = [()]
            continue
        paths = []
        for prev in prev_nodes[end]: 
            if prev is None:
                continue
            if (start, prev) not in cond_paths.keys(): # new case: when prev doesn't go back to start
                continue
            for sub_path in cond_paths[(start, prev)]:
                paths.append(sub_path + end[1:])
        cond_paths[(start, end)] = paths

    # new: add these paths to the cumulative all_paths
    for cond in cond_paths:
        all_paths[cond] = cond_paths[cond]
    
    # true probability
    for cond in cond_paths.keys(): 
        num = 0
        denom = 0 # new case: need a formula for denominator
        for path in cond_paths[cond]: # simply add up n() for all paths
            for prev_path in all_paths[((0,0), start)]: # new case: need earlier path
                num += n_symbol[prev_path + path]
        for prev_path in all_paths[((0,0), start)]:
            denom += n_symbol[prev_path]
        p_expr[cond] = sympy.simplify(num / denom) 

    # Markov probability
    for end in sorted(nodes): # use BFS
        if end[0] < start[0]:   
            continue
        if end[0] == start[0]:
            if end == start:    
                pr_expr[(start, end)] = 1
            continue
        expr = 0
        for prev in prev_nodes[end]:
            if prev is None:
                continue
            if (start, prev) not in cond_paths.keys(): # new case: when prev doesn't go back to start
                continue
            k, s = prev
            i, t = s2r(k+1)
            # Using Markov transition probability
            expr += pr_expr[(start, prev)] * q_expr[(i,t,s)][end[1] != s]
        pr_expr[(start, end)] = sympy.simplify(expr)

    # adjustment
    for cond in cond_paths.keys():
        a_expr[cond] = sympy.simplify(p_expr[cond] - pr_expr[cond])

# Take 10 seconds

In [19]:
# Step 3': check the result of Step 3

# If p_expr[cond] = 0 then pr_expr[cond] = 0 (and also a_expr[cond] = 0)
# Nothing should be printed
for cond in p_expr.keys():
    if p_expr[cond] == 0:
        if pr_expr[cond] != 0:
            display(pr_expr[cond])

# Print out an example formula
cond = ((2, 0), (4, 0))
display(sympy.Eq(p_symbol[cond], p_expr[cond]))
display(sympy.Eq(pr_symbol[cond], pr_expr[cond]))
display(sympy.Eq(a_symbol[cond], a_expr[cond]))

Eq(P(d_{2}=0;d_{4}=0), (n(0;0;0;0) + n(0;0;1;0) + n(1;0;0;0) + n(1;0;1;0))/(n(0;0) + n(1;0)))

Eq(Pr(d_{2}=0;d_{4}=0), ((n(0;0;0) + n(1;0;0))*(n(0;-1;0;0) + n(0;0;0;0) + n(1;0;0;0))*(n(0;0;1) + n(1;0;1) + n(1;1;1)) + (n(0;0;1) + n(1;0;1))*(n(0;-1;0) + n(0;0;0) + n(1;0;0))*(n(0;0;1;0) + n(1;0;1;0) + n(1;1;1;0)))/((n(0;0) + n(1;0))*(n(0;-1;0) + n(0;0;0) + n(1;0;0))*(n(0;0;1) + n(1;0;1) + n(1;1;1))))

Eq(A(d_{2}=0;d_{4}=0), (-(n(0;0;0) + n(1;0;0))*(n(0;-1;0;0) + n(0;0;0;0) + n(1;0;0;0))*(n(0;0;1) + n(1;0;1) + n(1;1;1)) - (n(0;0;1) + n(1;0;1))*(n(0;-1;0) + n(0;0;0) + n(1;0;0))*(n(0;0;1;0) + n(1;0;1;0) + n(1;1;1;0)) + (n(0;-1;0) + n(0;0;0) + n(1;0;0))*(n(0;0;1) + n(1;0;1) + n(1;1;1))*(n(0;0;0;0) + n(0;0;1;0) + n(1;0;0;0) + n(1;0;1;0)))/((n(0;0) + n(1;0))*(n(0;-1;0) + n(0;0;0) + n(1;0;0))*(n(0;0;1) + n(1;0;1) + n(1;1;1))))

### Analyse the adjustment

In [20]:
# Check 1: Adjustment equal 0 if and only if
# (i) the path start from (i, s_i) = (0, 0)
# (ii) their is an unique path to get from (i, s_i) to (j, s_j)
for cond in a_expr.keys():
    if p_expr[cond] != 0 and a_expr[cond] == 0:
        display(sympy.Eq(a_symbol[cond], a_expr[cond]))
        print()
        start, end = cond
        if start == (0,0):
            print(f"This path starts from (0,0)")
        else:
            if len(all_paths[cond]) == 1:
                print(f"There is an unique path from {start} to {end}")
        print("*"*10)

Eq(A(d_{0}=0;d_{0}=0), 0)


This path starts from (0,0)
**********


Eq(A(d_{0}=0;d_{1}=0), 0)


This path starts from (0,0)
**********


Eq(A(d_{0}=0;d_{1}=1), 0)

...


There is an unique path from (1, 1) to (2, 1)
**********


Eq(A(d_{1}=1;d_{3}=2), 0)


There is an unique path from (1, 1) to (3, 2)
**********


Eq(A(d_{4}=0;d_{4}=0), 0)


There is an unique path from (4, 0) to (4, 0)
**********


### Populate $n$'s using real data

In [21]:
# initialize n_val
n_val = {}
for seq in n_symbol.keys():
    n_val[seq] = 0

In [22]:
# get data
pso = pd.read_csv("./drive/MyDrive/data/final/pso.csv")
pso.head()

,game_id,kick_number,is_home_team,player_id,is_score
0,FAC_2020_3491225,1,True,351261,False
1,FAC_2020_3491225,2,False,37920,True
2,FAC_2020_3491225,3,True,334804,True
3,FAC_2020_3491225,4,False,314366,False
4,FAC_2020_3491225,5,True,167393,True


In [23]:
# convert data into sequences of penalty outcomes
pso_outs = pso.groupby("game_id")["is_score"].apply(lambda x: tuple(map(int, x)))
pso_outs.head()

game_id
19YL_2014_2528464                (1, 0, 1, 0, 1, 1, 0, 0)
19YL_2014_2528466                   (0, 1, 1, 1, 0, 1, 0)
19YL_2014_2528467    (1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1)
19YL_2014_2528470    (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0)
19YL_2014_2542063          (0, 1, 1, 1, 1, 1, 1, 1, 1, 1)
Name: is_score, dtype: object

In [24]:
# populate n_val
for out in pso_outs:
    for i in range(min(len(out) + 1, 2*nround + 1)):
        diff = o2d(out[:i])
        if diff in n_val.keys():
            n_val[diff] += 1

In [25]:
# calculate transition probability
q_val = {}
for state in q_symbol.keys():
    q_val[state] = q_expr[state][1].evalf(subs={n_symbol[seq]: n_val[seq] for seq in n_symbol.keys()})

In [26]:
# print out the approximation
for state in sorted(q_symbol.keys(), key=lambda x: (x[1], x[0], -x[2])):
    display(sympy.Eq(q_symbol[state], q_val[state]))

Eq(q_{1;1}(0), 0.793866728153101)

Eq(q_{2;1}(1), 0.783038048213767)

Eq(q_{2;1}(0), 0.739373601789709)

Eq(q_{1;2}(1), 0.75609756097561)

Eq(q_{1;2}(0), 0.775803144224197)

Eq(q_{1;2}(-1), 0.733736762481089)

Eq(q_{2;2}(1), 0.776734693877551)

Eq(q_{2;2}(0), 0.756876663708962)